# Jaeger-query with trace DSL

In [4]:
%%loadFromPOM
<dependency>
    <groupId>com.fasterxml.jackson.core</groupId>
    <artifactId>jackson-databind</artifactId>
    <version>2.6.7</version>
</dependency>
<dependency>
    <groupId>com.fasterxml.jackson.core</groupId>
    <artifactId>jackson-annotations</artifactId>
    <version>2.6.7</version>
</dependency>
<dependency>
    <groupId>io.prometheus</groupId>
    <artifactId>simpleclient_httpserver</artifactId>
    <version>0.7.0</version>
</dependency>
    <dependency>
      <groupId>org.apache.tinkerpop</groupId>
      <artifactId>tinkergraph-gremlin</artifactId>
      <version>3.4.3</version>
    </dependency>
<dependency>
    <groupId>log4j</groupId>
    <artifactId>log4j</artifactId>
    <version>1.2.17</version>
</dependency>


In [1]:
%maven io.jaegertracing:jaeger-tracedsl:1.0-SNAPSHOT

System.out.println(io.jaegertracing.dsl.gremlin.Keys.class);

class io.jaegertracing.dsl.gremlin.Keys


In [2]:
import org.apache.tinkerpop.gremlin.structure.Graph;
import org.apache.tinkerpop.gremlin.structure.Vertex;
import io.jaegertracing.dsl.gremlin.*;
import io.jaegertracing.dsl.gremlin.ui.*;
import io.jaegertracing.dsl.gremlin.model.*;

Trace trace = JaegerQueryService.load("54e26ad4bbea6606", "http://192.168.122.1:16686/api/traces/");
Graph graph = GraphCreator.create(trace);
TraceDepth.calculate(graph);


EvalException: com/fasterxml/jackson/core/type/TypeReference